# Chat

[This example](https://gptcache.readthedocs.io/en/latest/bootcamp/openai/chat.html) will show you how to chat with GPT, the original example is on [OpenAI Example](https://platform.openai.com/docs/guides/chat/introduction), the difference is that we will teach you how to cache the  response for exact and similar matches with **gptcache**, it will be very simple, you just need to add an extra step to initialize the cache.


In [ ]:
! pip install -q gptcache

Before running the example, make sure the `OPENAI_API_KEY` environment variable is set by executing `echo $OPENAI_API_KEY`. If it is not already set, it can be set by using `export OPENAI_API_KEY=YOUR_API_KEY` on Unix/Linux/MacOS systems or `set OPENAI_API_KEY=YOUR_API_KEY` on Windows systems.

> We can run `os.environ` to set the environment variable in colab.

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

Then we can learn the usage and acceleration effect of gptcache by the following code, which consists of three parts, the original openai way, the exact search and the similar search.


## OpenAI API original usage

In [ ]:
import time
import openai


def response_text(openai_resp):
    return openai_resp['choices'][0]['message']['content']


question = 'what‘s github?'

# OpenAI API original usage
start_time = time.time()
response = openai.ChatCompletion.create(
  model='gpt-3.5-turbo',
  messages=[
    {
        'role': 'user',
        'content': question
    }
  ],
)
print(f'Question: {question}')
print("Time consuming: {:.2f}s".format(time.time() - start_time))
print(f'Answer: {response_text(response)}\n')

Question: what‘s github?
Time consuming: 5.07s
Answer: GitHub is an online platform designed for version control and collaboration on software project repositories by providing tools such as source code management, issue tracking, and project management functions. GitHub allows developers to store and manage their code remotely and collaborate with other contributors on a project. It also serves as a platform for hosting and sharing open-source projects.



## OpenAI API + GPTCache, exact match cache

Initalize the cache to run GPTCache and import `openai` form `gptcache.adapter`, which will automatically set the map data manager to match the exact cahe, more details refer to [build your cache](https://gptcache.readthedocs.io/en/dev/usage.html#build-your-cache).

And if you ask ChatGPT the exact same two questions, the answer to the second question will be obtained from the cache without requesting ChatGPT again.

In [ ]:
import time


def response_text(openai_resp):
    return openai_resp['choices'][0]['message']['content']

print("Cache loading.....")

# To use GPTCache, that's all you need
# -------------------------------------------------
from gptcache import cache
from gptcache.adapter import openai

cache.init()
cache.set_openai_key()
# -------------------------------------------------

question = "what's github"
for _ in range(2):
    start_time = time.time()
    response = openai.ChatCompletion.create(
      model='gpt-3.5-turbo',
      messages=[
        {
            'role': 'user',
            'content': question
        }
      ],
    )
    print(f'Question: {question}')
    print("Time consuming: {:.2f}s".format(time.time() - start_time))
    print(f'Answer: {response_text(response)}\n')

Cache loading.....
Question: what's github
Time consuming: 6.04s
Answer: GitHub is an online platform where developers can share and collaborate on software development projects. It is used as a hub for code repositories and includes features such as issue tracking, code review, and project management tools. GitHub can be used for open source projects, as well as for private projects within organizations. GitHub has become an essential tool within the software development industry and has over 40 million registered users as of 2021.

Question: what's github
Time consuming: 0.00s
Answer: GitHub is an online platform where developers can share and collaborate on software development projects. It is used as a hub for code repositories and includes features such as issue tracking, code review, and project management tools. GitHub can be used for open source projects, as well as for private projects within organizations. GitHub has become an essential tool within the software development in

## OpenAI API + GPTCache, similar search cache

Set the cache with `embedding_func` to generate embedding for the text, and `data_manager` to manager the cache data, `similarity_evaluation` to evaluate the similarities, more details refer to [build your cache](https://gptcache.readthedocs.io/en/dev/usage.html#build-your-cache).

After obtaining an answer from ChatGPT in response to several similar questions, the answers to subsequent questions can be retrieved from the cache without the need to request ChatGPT again.

In [ ]:
import time


def response_text(openai_resp):
    return openai_resp['choices'][0]['message']['content']

from gptcache import cache
from gptcache.adapter import openai
from gptcache.embedding import Onnx
from gptcache.manager import CacheBase, VectorBase, get_data_manager
from gptcache.similarity_evaluation.distance import SearchDistanceEvaluation

print("Cache loading.....")

onnx = Onnx()
data_manager = get_data_manager(CacheBase("sqlite"), VectorBase("faiss", dimension=onnx.dimension))
cache.init(
    embedding_func=onnx.to_embeddings,
    data_manager=data_manager,
    similarity_evaluation=SearchDistanceEvaluation(),
    )
cache.set_openai_key()

questions = [
    "what's github",
    "can you explain what GitHub is",
    "can you tell me more about GitHub",
    "what is the purpose of GitHub"
]

for question in questions:
    start_time = time.time()
    response = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        messages=[
            {
                'role': 'user',
                'content': question
            }
        ],
    )
    print(f'Question: {question}')
    print("Time consuming: {:.2f}s".format(time.time() - start_time))
    print(f'Answer: {response_text(response)}\n')

Cache loading.....
Question: what's github
Time consuming: 2.89s
Answer: GitHub is a web-based platform that is used for version control and collaboration on software development projects. It was launched in 2008 and has since become a popular platform for open-source and closed-source software projects. Users can store their code on GitHub, track changes made by other contributors, suggest changes, and collaborate on code together without physically being in the same place. It has become a hub for developers to share their work and contribute to other projects, making it an important tool for software development.

Question: can you explain what GitHub is
Time consuming: 1.42s
Answer: GitHub is a web-based platform that is used for version control and collaboration on software development projects. It was launched in 2008 and has since become a popular platform for open-source and closed-source software projects. Users can store their code on GitHub, track changes made by other contri